In [ ]:
from google.colab import drive

drive.mount('/content/drive')

%cd drive/MyDrive/Colab Notebooks
#! ls

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/Colab Notebooks


In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('mathnation_sample2.csv').sample(frac=1)
#df_train = df.iloc[:9000]
#df_val = df.iloc[9000:]
#train_dataset = Dataset.from_pandas(df_train)
#val_dataset = Dataset.from_pandas(df_val)

In [ ]:
df_trainA = df.sample(frac=0.8, random_state=44)
df_valA = df.drop(df_trainA.index).reset_index(drop=True)
df_trainA = df_trainA.reset_index(drop=True)

In [ ]:
df_trainA.head()

,id,session_log_id,useraccount_id,subject_id,counter,class_id,school_id,is_parent_post,reply_to_post_id,karma_awarded_id,...,access_permissions,answer_status_flags,ts_created,ts_modified,TOXICITY,IDENTITY_ATTACK,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,206793,381972.0,1777832,1,207789,NaN,13669,0,206742.0,NaN,...,4294967295,0,5/1/2014 0:18,5/1/2014 0:18,1,0,1,1,0,0
1,310,NaN,844235,1,1309,NaN,11646,0,283.0,NaN,...,4294967295,0,41562.82345,41562.82345,0,0,0,0,0,0
2,193,NaN,482661,1,1192,NaN,11576,0,152.0,NaN,...,4294967295,0,41562.66274,41562.66274,0,0,0,0,0,0
3,276,NaN,828691,1,1275,NaN,11646,0,273.0,NaN,...,4294967295,0,41562.81963,41562.81963,0,0,0,0,0,0
4,212594,292154.0,1580563,1,213590,NaN,13273,0,212554.0,NaN,...,4294967295,0,5/3/2014 13:14,5/3/2014 13:14,1,0,1,1,0,0


In [ ]:
df_train = df_trainA[['id', 'comment_text', 'TOXICITY', 'IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'THREAT', 'SEXUALLY_EXPLICIT']]
df_val = df_valA[['id', 'comment_text', 'TOXICITY', 'IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'THREAT', 'SEXUALLY_EXPLICIT']]
df_train.head()

,id,comment_text,TOXICITY,IDENTITY_ATTACK,INSULT,PROFANITY,THREAT,SEXUALLY_EXPLICIT
0,206793,stupid people these days( ment for me not you ...,1,0,1,1,0,0
1,310,me to me too me too \n,0,0,0,0,0,0
2,193,thanks,0,0,0,0,0,0
3,276,OOH KARMA POINTS I see it now,0,0,0,0,0,0
4,212594,\nInformal. an utterly foolish or senseless p...,1,0,1,1,0,0


In [ ]:
label_cols = ['TOXICITY', 'IDENTITY_ATTACK', 'INSULT', 'PROFANITY', 'THREAT', 'SEXUALLY_EXPLICIT']

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Setting up the device for GPU usage
from torch import cuda

device = 'cuda' if cuda.is_available() else 'cpu'

In [ ]:
import torch
import transformers
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DistilBertModel, DistilBertTokenizer

In [ ]:
model_name = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(model_name, do_lower_case=True)

In [ ]:
# hyperparameters
MAX_LEN = 512
TRAIN_BATCH_SIZE = 12
VALID_BATCH_SIZE = 12
EPOCHS = 2
LEARNING_RATE = 1e-05

## Custom Dataset

In [ ]:
class ToxicDataset(Dataset):
    def __init__(self, data, tokenizer, max_len):
        # Initialize the attribute
        self.data = data # The dataset object
        self.tokenizer = tokenizer
        self.max_len = max_len
        self.labels = self.data[label_cols].values

    def __len__(self):
        # Return the size of the dataset
        return len(self.data.id)

    def __getitem__(self, idx):
        # Return the i-th sample
        # Get the image and the label from the dataset object
        text = self.data.comment_text
        tokenized_text = self.tokenizer.encode_plus(
            str( text ),
            None,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            return_token_type_ids=True,
            truncation=True,
            return_attention_mask=True,
            return_tensors='pt'
        )

        return {
            'input_ids': tokenized_text['input_ids'].flatten(),
            'attention_mask': tokenized_text['attention_mask'].flatten(),
#             'token_type_ids': tokenized_text["token_type_ids"].flatten(),
            'targets': torch.FloatTensor(self.labels[idx])
        }

In [ ]:
train_dataset = ToxicDataset(df_train, tokenizer, MAX_LEN)
valid_dataset = ToxicDataset(df_val, tokenizer, MAX_LEN)

In [ ]:
train_data_loader = torch.utils.data.DataLoader(train_dataset,
    batch_size=TRAIN_BATCH_SIZE,
    shuffle=True,
    num_workers=0
)

val_data_loader = torch.utils.data.DataLoader(valid_dataset,
    batch_size=VALID_BATCH_SIZE,
    shuffle=False,
    num_workers=0
)

In [ ]:
# from transformers import DistilBertForSequenceClassification, AdamW
# model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased')
# model.to(device)
# print()

In [ ]:
# from transformers import DistilBertModel

class CustomDistilBertClass(torch.nn.Module):
    def __init__(self):
        super(CustomDistilBertClass, self).__init__()
        self.distilbert_model = DistilBertModel.from_pretrained(model_name, return_dict=True)
        self.dropout = torch.nn.Dropout(0.3)
        self.linear = torch.nn.Linear(768, 6)

    def forward(self, input_ids, attn_mask):
        output = self.distilbert_model(
            input_ids,
            attention_mask=attn_mask,
#             token_type_ids=token_type_ids
        )
        output_dropout = self.dropout(output.last_hidden_state)
        output = self.linear(output_dropout)
        return output

In [ ]:
model = CustomDistilBertClass()
model.to(device)
print()

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
def loss_fn(outputs, targets):
    return torch.nn.BCEWithLogitsLoss()(outputs, targets)
######### !! CHECK HERE ! ###########

optimizer = torch.optim.Adam(params =  model.parameters(), lr=LEARNING_RATE)

In [ ]:
def train_model(n_epochs, training_loader, validation_loader, model,
                optimizer, checkpoint_path, best_model_path):

  # initialize tracker for minimum validation loss
  valid_loss_min = np.Inf

  for epoch in range(1, n_epochs+1):
    train_loss = 0
    valid_loss = 0

    model.train()
    print('############# Epoch {}: Training Start   #############'.format(epoch))
    for batch_idx, data in enumerate(training_loader):
#         print(f"batch_idx: {batch_idx}", sep=' ')
        ids = data['input_ids'].to(device, dtype = torch.long)
        mask = data['attention_mask'].to(device, dtype = torch.long)

        outputs = model(ids, mask, )
        outputs = outputs[:, 0, :]
        targets = data['targets'].to(device, dtype = torch.float)
        loss = loss_fn(outputs, targets)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss = train_loss + ((1 / (batch_idx + 1)) * (loss.item() - train_loss))

    print('############# Epoch {}: Training End     #############'.format(epoch))

    print('############# Epoch {}: Validation Start   #############'.format(epoch))
    ######################
    # validate the model #
    ######################

    model.eval()

    with torch.no_grad():
      for batch_idx, data in enumerate(validation_loader, 0):
            ids = data['input_ids'].to(device, dtype = torch.long)
            mask = data['attention_mask'].to(device, dtype = torch.long)

            targets = data['targets'].to(device, dtype = torch.float)
            outputs = model(ids, mask, )
            outputs = outputs[:, 0, :]
            loss = loss_fn(outputs, targets)

            valid_loss = valid_loss + ((1 / (batch_idx + 1)) * (loss.item() - valid_loss))

      print('############# Epoch {}: Validation End     #############'.format(epoch))
      train_loss = train_loss/len(training_loader)
      valid_loss = valid_loss/len(validation_loader)
      # print training/validation statistics
      print('Epoch: {} \tAvgerage Training Loss: {:.6f} \tAverage Validation Loss: {:.6f}'.format(
            epoch,
            train_loss,
            valid_loss
            ))

      # create checkpoint variable and add important data
      checkpoint = {
            'epoch': epoch + 1,
            'valid_loss_min': valid_loss,
            'state_dict': model.state_dict(),
            'optimizer': optimizer.state_dict()
      }

        # save checkpoint
      save_ckp(checkpoint, False, checkpoint_path, best_model_path)

      ## TODO: save the model if validation loss has decreased
      if valid_loss <= valid_loss_min:
        print('Validation loss decreased ({:.6f} --> {:.6f}).  Saving model ...'.format(valid_loss_min,valid_loss))
        # save checkpoint as best model
        save_ckp(checkpoint, True, checkpoint_path, best_model_path)
        valid_loss_min = valid_loss

    print('############# Epoch {}  Done   #############\n'.format(epoch))

  return model

In [ ]:
import shutil

def load_ckp(checkpoint_fpath, model, optimizer):
    """
    checkpoint_path: path to save checkpoint
    model: model that we want to load checkpoint parameters into
    optimizer: optimizer we defined in previous training
    """
    # load check point
    checkpoint = torch.load(checkpoint_fpath)
    # initialize state_dict from checkpoint to model
    model.load_state_dict(checkpoint['state_dict'])
    # initialize optimizer from checkpoint to optimizer
    optimizer.load_state_dict(checkpoint['optimizer'])
    # initialize valid_loss_min from checkpoint to valid_loss_min
    valid_loss_min = checkpoint['valid_loss_min']
    # return model, optimizer, epoch value, min validation loss
    return model, optimizer, checkpoint['epoch'], valid_loss_min.item()

def save_ckp(state, is_best, checkpoint_path, best_model_path):
    """
    state: checkpoint we want to save
    is_best: is this the best checkpoint; min validation loss
    checkpoint_path: path to save checkpoint
    best_model_path: path to save best model
    """
    f_path = checkpoint_path
    # save checkpoint data to the path given, checkpoint_path
    torch.save(state, f_path)
    # if it is a best model, min validation loss
    if is_best:
        best_fpath = best_model_path
        # copy that checkpoint file to best path given, best_model_path
        shutil.copyfile(f_path, best_fpath)

In [ ]:
ckpt_path = "./model.pt"
best_model_path = "./best_model.pt"

In [ ]:

trained_model = train_model(EPOCHS,
                            train_data_loader,
                            val_data_loader,
                            model,
                            optimizer,
                            ckpt_path,
                            best_model_path)

############# Epoch 1: Training Start   #############
############# Epoch 1: Training End     #############
############# Epoch 1: Validation Start   #############
############# Epoch 1: Validation End     #############
Epoch: 1 	Avgerage Training Loss: 0.020546 	Average Validation Loss: 0.071292
Validation loss decreased (inf --> 0.071292).  Saving model ...
############# Epoch 1  Done   #############

############# Epoch 2: Training Start   #############
############# Epoch 2: Training End     #############
############# Epoch 2: Validation Start   #############
############# Epoch 2: Validation End     #############
Epoch: 2 	Avgerage Training Loss: 0.016848 	Average Validation Loss: 0.072425
############# Epoch 2  Done   #############

